In [1]:
from Evaluation.Evaluator import EvaluatorHoldout
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

URM_all_dataframe = pd.read_csv(filepath_or_buffer="1OUR_NOTEBOOKS/data_train.csv")
ICM_all = pd.read_csv(filepath_or_buffer="1OUR_NOTEBOOKS/data_ICM_metadata.csv")
URM_all_dataframe.head(n=10)


URM_train_validation, URM_test = train_test_split(URM_all_dataframe, test_size=0.2, random_state=42)
URM_train, URM_validation = train_test_split(URM_train_validation, test_size=0.2, random_state=42)

# Reset indices for cleaner output (optional)
URM_test = URM_test.reset_index(drop=True)
URM_validation = URM_validation.reset_index(drop=True)
URM_train = URM_train.reset_index(drop=True)
URM_train_validation = URM_train_validation.reset_index(drop=True)

# Display the results
print("URM_train (80% of URM_ALL):")
print(URM_train.head(n=10))

print("\nURM_validation (20% of URM_ALL):")
print(URM_validation.head(n=10))

print("\nURM_test (20% of URM_ALL):")
print(URM_test.head(n=10))


URM_train (80% of URM_ALL):
   user_id  item_id  data
0     1523    11379   1.0
1     3415    29647   1.0
2     6571    11101   1.0
3    31036    17962   1.0
4    22131    20291   1.0
5    25670    32878   1.0
6     6677    30720   1.0
7    23612    35978   1.0
8    17411     9221   1.0
9    16720     2668   1.0

URM_validation (20% of URM_ALL):
   user_id  item_id  data
0    23345    37683   1.0
1    16471     1559   1.0
2    19988     6956   1.0
3     8833      614   1.0
4    23100    20305   1.0
5     7748      117   1.0
6    22148    35968   1.0
7    29212    33031   1.0
8    17563    34624   1.0
9     9451     9732   1.0

URM_test (20% of URM_ALL):
   user_id  item_id  data
0    18939     8673   1.0
1     4669     2436   1.0
2    20972    25295   1.0
3    22394    23927   1.0
4     3680    24294   1.0
5    24013    14508   1.0
6     7203     5330   1.0
7    34271    35125   1.0
8     7511     8894   1.0
9    17839    20469   1.0


In [2]:
URM_all_dataframe = csr_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_train = csr_matrix((URM_train['data'], (URM_train['user_id'], URM_train['item_id'])))
URM_validation = csr_matrix((URM_validation['data'], (URM_validation['user_id'], URM_validation['item_id'])))
URM_test = csr_matrix((URM_test['data'], (URM_test['user_id'], URM_test['item_id'])))
URM_train_validation = csr_matrix((URM_train_validation['data'], (URM_train_validation['user_id'], URM_train_validation['item_id'])))
ICM_all = csr_matrix((ICM_all['data'], (ICM_all['item_id'], ICM_all['feature_id'])))

#evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 401 ( 1.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 152 ( 0.4%) Users that have less than 1 test interactions


In [ ]:
import optuna
from Recommenders.SLIM.MSLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender

# Definizione della funzione obiettivo per Optuna
def objective(trial):
    # Definizione dei parametri da ottimizzare
    topK = trial.suggest_int("topK", 100, 200)  # Intervallo tra 1 e 1000
    
    # Creazione dell'istanza del recommender
    recommender_instance = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train_validation)
    
    recommender_instance.fit(
        topK=topK,
        l1_ratio=0.49705634165029133, 
        alpha=9.773610229057873e-05,
        positive_only=True,  # Manteniamo fisso questo parametro
    )
    
    # Valutazione delle prestazioni del modello
    result_df, _ = evaluator_test.evaluateRecommender(recommender_instance)
    map_score = result_df.loc[10]["MAP"]  # Estrarre il valore di MAP per k=10
    print(f"MAP ottenuto: {map_score} (topK={topK})")

    return map_score

# Creazione dello studio Optuna
study = optuna.create_study(direction="maximize")  # Maximizzare MAP
study.optimize(objective, n_trials=50)  # Numero di tentativi

# Migliori parametri trovati
best_params = study.best_params
best_score = study.best_value

print(f"Migliori parametri trovati: {best_params}")
print(f"MAP ottenuto: {best_score}")


[I 2024-12-01 14:00:49,612] A new study created in memory with name: no-name-3390f014-3d1d-47e0-9d34-233783c757b3
Progress: 100%|██████████| 38121/38121 [14:19<00:00, 44.36items/s] 


EvaluatorHoldout: Processed 35584 (100.0%) in 12.64 sec. Users per second: 2815


[I 2024-12-01 14:15:21,895] Trial 0 finished with value: 0.0607631640223513 and parameters: {'topK': 170}. Best is trial 0 with value: 0.0607631640223513.
Progress:   0%|          | 0/38121 [00:00<?, ?items/s]

MAP ottenuto: 0.0607631640223513 (topK=170)


Progress: 100%|██████████| 38121/38121 [14:27<00:00, 43.94items/s] 


...

Progress: 100%|██████████| 38121/38121 [14:24<00:00, 44.07items/s] 


EvaluatorHoldout: Processed 35584 (100.0%) in 12.79 sec. Users per second: 2781


[I 2024-12-01 18:53:55,732] Trial 19 finished with value: 0.06077718182596552 and parameters: {'topK': 182}. Best is trial 16 with value: 0.060779409954177244.
Progress:   0%|          | 0/38121 [00:00<?, ?items/s]

MAP ottenuto: 0.06077718182596552 (topK=182)


Progress:   0%|          | 0/38121 [10:52<?, ?items/s]
